<center><img src="https://www.pctleganes.org/images/asociados/uc3m.jpg" alt="UC3M Banner"/></center>

---
# PRÁCTICA 2: PREDICCIÓN DEL ABANDONO (BURNOUT) DE EMPLEADOS

**Realizado por:**
| NIA | NOMBRE |
| ------ | ------ |
| 100432074 | Diego Alonso Herreruela | 
| 100432274 | Antonio Gómez Otero |
---

# MODELO FINAL

## Introducción

En estos apartados se va a desarrollar el **modelo final de predicción de abandono de clientes** utilizando la técnica de selección de características por filtrado. Basándonos en los resultados obtenidos en la práctica anterior, se ha decidido utilizar el modelo HistGradientBoosting debido a su buen rendimiento en términos de precisión y tiempo de ejecución. Ahora, con la implementación de la selección de características, se espera mejorar aún más el rendimiento del modelo final. En este archivo se llevará a cabo la creación del modelo, su ajuste y finalmente se generarán las predicciones para la competición.

In [1]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np
import pickle

# Control de tiempos
# ==============================================================================
import time

# Preprocesado
# ==============================================================================
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Gradient Boosting
# ==============================================================================
from sklearn.ensemble import HistGradientBoostingClassifier

# Pipeline
# ==============================================================================
from sklearn.pipeline import Pipeline

Volvemos a tratar los datos que habíamos visto anteriormente.

In [2]:
with open("attrition_available_22.pkl", 'rb') as file:
    disp_df = pickle.load(file)
disp_df.nunique()

hrs                        4406
absences                     24
JobInvolvement                4
PerformanceRating             2
EnvironmentSatisfaction       4
JobSatisfaction               4
WorkLifeBalance               4
Age                          43
Attrition                     2
BusinessTravel                3
Department                    3
DistanceFromHome             29
Education                     5
EducationField                6
EmployeeCount                 1
EmployeeID                 3644
Gender                        2
JobLevel                      5
JobRole                       9
MaritalStatus                 3
MonthlyIncome              1341
NumCompaniesWorked           10
Over18                        1
PercentSalaryHike            15
StandardHours                 1
StockOptionLevel              4
TotalWorkingYears            40
TrainingTimesLastYear         7
YearsAtCompany               37
YearsSinceLastPromotion      16
YearsWithCurrManager         18
dtype: i

---
## Creación del modelo final

Aquí volvemos a presentar el desbalanceo en la variable de salida.

In [3]:
X_df = disp_df.drop('Attrition', axis=1)
attri_count = disp_df['Attrition'].value_counts()
y_df = disp_df.Attrition
y_df.value_counts()

No     3699
Yes     711
Name: Attrition, dtype: int64

Aquí procedemos a quitar las variables que hemos considerado menos relevantes.

In [4]:
# Borrado de las variables que no interesan
X_df.drop(['EmployeeCount', 'EmployeeID', 'Gender', 'StandardHours', 'Over18', 'DistanceFromHome', 'Department', 'PerformanceRating'], axis=1, inplace=True)
print('La forma de la tabla original, disp_df, es de:')
print('===============================')
print(str(len(disp_df.index)) + ' filas y ' + str(len(disp_df.columns)) + ' columnas')
print()
print('La forma de la tabla modificada, X_df (también sin Attrition), es de:')
print('===============================')
print(str(len(X_df.index)) + ' filas y ' + str(len(X_df.columns)) + ' columnas')
print()

La forma de la tabla original, disp_df, es de:
4410 filas y 31 columnas

La forma de la tabla modificada, X_df (también sin Attrition), es de:
4410 filas y 22 columnas



Entrenamos y aplicamos el preprocesador.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, stratify=y_df, test_size=0.2)

In [6]:
le = preprocessing.LabelEncoder()
le.fit(y_df)

y_test = le.transform(y_test)
y_train = le.transform (y_train)

In [7]:
num_cols = X_train.select_dtypes(exclude="object").columns
cat_cols = X_train.select_dtypes(include="object").columns
X_train[cat_cols] = X_train[cat_cols].astype('category')

In [8]:
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
    ('pca', PCA(n_components=5))
])

num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', RobustScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols)
    ])

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', KNNImputer()),
                                                 ('scaler', RobustScaler())]),
                                 Index(['hrs', 'absences', 'JobInvolvement', 'EnvironmentSatisfaction',
       'JobSatisfaction', 'WorkLifeBalance', 'Age', 'Education', 'JobLevel',
       'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimes...ear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False)),
                                                 ('pca', PCA(n_components=5))]),
                                 Index(['BusinessTravel', 'EducationField', 'JobRole', 'MaritalStatus'], dtype='object'))])

## HistGradientBoosting con filtrado

In [9]:
# Aquí, establecemos nuestro modelo como HistGradientBoostingRegressor
modelo_final_gb = HistGradientBoostingClassifier(
                                                class_weight='balanced', 
                                                l2_regularization=0,
                                                learning_rate=0.4, 
                                                max_depth=36,
                                                max_leaf_nodes=33, 
                                                min_samples_leaf=9,
                                                random_state=22)


# Hacemos el preprocesado y preparamos el modelo en un pipeline
modelo_final = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', modelo_final_gb)])


np.random.seed(22) # Reproductibilidad

# Lo entrenamos
tiempo_inicio = time.time()
modelo_final.fit(X_train, y_train)
tiempo_fin = time.time()

# Tiempo de ejecución
t_mf = tiempo_fin - tiempo_inicio
print('===========================================================')
print("El tiempo de entrenamiento del modelo es", t_mf, 'segundos')

El tiempo de entrenamiento del modelo es 3.031682014465332 segundos


Finalmente, lo guardamos en un fichero .plk y, además, generamos un csv con las predicciones que se tomarían al aplicar nuestro modelo.

In [10]:
# Para guardar el modelo en un fichero .plk
pkl_filename = "modelo_final.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(modelo_final, file)

In [12]:
# Para guardar las predicciones en un fichero .csv
predicciones = modelo_final.predict(X_test)

csv_filename = "predicciones.csv"

fichero_predicciones = pd.DataFrame(predicciones, columns = ['Predicciones']).to_csv(csv_filename, index=False, header=False)